In [1]:
!pip install --quiet yahoo-fin

     |████████████████████████████████| 81 kB 3.4 MB/s 
     |████████████████████████████████| 83 kB 2.0 MB/s 
     |████████████████████████████████| 127 kB 7.7 MB/s 
     |████████████████████████████████| 103 kB 42.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from yahoo_fin import options,stock_info
import numpy as np
import pandas as pd 
from datetime import datetime,timedelta
from collections import defaultdict
from tqdm import tqdm
from typing import List

## Q1 

- Consider the gains of the following stocks from January 2nd, 2021 until December 1st 2021
  -	Apple Computers (AAPL)
  -	Microsoft (MSFT)
  -	Meta (Formerly Face Book) (FB
  - Alphabet (GOOG)


- Suppose you want to preserve 50% of the above gains until June 17, 2022. Find the strike and premium of a put option that accomplishes this

(To determine premium: use use simple linear interpolation on the ask prices of exchange traded options)

## Q2

Determine the strike of a call that selling it will make the collar zero cost. (buying the put and selling the call). Use linear interpolation on the bid price of the calls. 

## Q3
If we only sell the call (Covered call) and the stock is called on the maturity of the option, what would be the return from January 2nd, 2021 until the call date

 

In [3]:
def compute_stock_return(ticker:str, start_date:datetime,end_date:datetime)->float:
  """
  Function that returns the gains obtained on investment of 1 unit of a particular stock within the stipulated time period
  """
  stock_df = stock_info.get_data(ticker, start_date, end_date)
  return stock_df["close"][-1] - stock_df["close"][0]

In [4]:
def compute_strike_price(ticker:str,spot_date:datetime, g: float)->float:
  """
  Method to compute the strike of the put option
  """
  # Get the Spot Price
  spot_price = stock_info.get_data(ticker, spot_date)["close"][0]
  return spot_price - g

In [5]:
def find_nearest_indices(df,colname,val):
    """
    From a given dataframe obtain the index of the required key in a particular column or the indices of the nearest neighbours
    """
    if val in df[colname].values:
        return df.index[df[colname]==val].tolist()
    else:
        upper_i = df[df[colname] < val][colname].idxmax()
        lower_i = df[df[colname] > val][colname].idxmin()
        return [upper_i, lower_i]

In [6]:
def get_interplolated_val(x1:float, y1:float, x2:float, y2:float, x:float)-> float:
  m = (y2-y1)/(x2-x1)
  y = m*(x-x1) + y1
  return y

In [7]:
def compute_put_premium(ticker:str, strike_p: float, maturity:str ) -> float:
  # Fetch the Option Data
  option_df = options.get_options_chain(ticker, maturity)
  # Slice the Put Data
  puts_df = option_df["puts"]
  # Fetch the index/indices of the nearest ask price
  ask_i = find_nearest_indices(puts_df,"Strike",strike_p)
  # If the exact ask price is found
  if len(ask_i) == 1:
    # Return it as the premium
    premium = puts_df.loc[ask_i[0],["Last Price"]][0]
  else:
    x1, y1 = puts_df.loc[ask_i[0],["Strike","Last Price"]]
    x2, y2 = puts_df.loc[ask_i[1],["Strike","Last Price"]]
    x = strike_p
    premium = get_interplolated_val(x1,y1,x2,y2,x)
  return premium

In [8]:
def compute_call_strike(ticker:str, p: float, maturity:str ) -> float:
  # Fetch the Option Data
  option_df = options.get_options_chain(ticker, maturity)
  # Slice the Put Data
  calls_df = option_df["calls"]
  # Fetch the index/indices of the nearest ask price
  ask_i = find_nearest_indices(calls_df,"Last Price",p)
  # If the exact ask price is found
  if len(ask_i) == 1:
    # Return it as the premium
    strike = calls_df.loc[ask_i[0],["Strike"]][0]
  else:
    x1, y1 = calls_df.loc[ask_i[0],["Last Price","Strike"]]
    x2, y2 = calls_df.loc[ask_i[1],["Last Price","Strike"]]
    x = p
    strike = get_interplolated_val(x1,y1,x2,y2,x)
  return strike

In [9]:
def compute_covered_call_r(ticker:str, start_date:datetime, strike_p: float):
  initial_p = stock_info.get_data(ticker, start_date)["close"][0]
  return strike_p - initial_p 

In [11]:
def get_strike_premium(tickers:List[str] ,start_date:datetime ,end_date:datetime ,spot_date:datetime, maturity_date:datetime)->pd.DataFrame:
  """
  Function that returns a dataframe for the respective gains and desired put strike prices for a given list of stocks to save 50% of the gains earned within a period
  """
  portfolio = defaultdict(dict)
  for t in tqdm(tickers):
    # Compute the returns 
    portfolio[t]["Gains"] = compute_stock_return(t,start_date,end_date)
    # Get the Required Strike Price for Put
    portfolio[t]["Put Strike"] = compute_strike_price(t,spot_date,(portfolio[t]["Gains"])/2)
    # Compute the Required Premium for Put
    portfolio[t]["Put Premium"] = compute_put_premium(t,portfolio[t]["Put Strike"],maturity_date)
    # For a Zero Collar, we need to find a Call of the same premium
    portfolio[t]["Call Premium"] = portfolio[t]["Put Premium"]
    # Use the Premium to find strike of a Call
    portfolio[t]["Call Strike"] = compute_call_strike(t,portfolio[t]["Call Premium"],maturity_date)
    # USe Call Strije to find covered Call Return
    portfolio[t]["Covered Call Return"] = compute_covered_call_r(t,start_date,portfolio[t]["Call Strike"])
  # Convert the result into a dataframe and return it
  return pd.DataFrame.from_dict(portfolio,orient="index")

In [12]:
start_date = datetime(2021,1,2)
end_date = datetime(2021,12,2)
spot_date = datetime(2021,12,10)
maturity_date = datetime(2022,6,17)
tickers = ["AAPL", "MSFT","GOOGL","FB"]

In [13]:
portfolio_df = get_strike_premium(tickers,start_date,end_date,spot_date,maturity_date)
portfolio_df

100%|██████████| 4/4 [00:14<00:00,  3.62s/it]


,Gains,Put Strike,Put Premium,Call Premium,Call Strike,Covered Call Return
AAPL,35.360001,161.769997,11.011099,11.011099,180.469171,51.059167
MSFT,112.389984,286.345016,13.903505,13.903505,350.554984,132.864981
GOOGL,1094.900024,2412.580017,72.447131,72.447131,3318.342072,1592.212068
FB,41.660004,308.919998,22.184399,22.184399,363.358097,94.418094
